# OAS -> FC
> This notebook is to validate OAS -> FC (Function-Calling) conversion and implement python module for that.

In detail, https://chatgpt.com/share/6720630a-827c-8002-85f7-6f02eff3d382

In [1]:
#| default_exp ninjalabo.llmcam.oas_to_fc

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
from fastapi import FastAPI
from typing import Optional
import uvicorn
import threading
import httpx
import time
import json
import openai

In [5]:
app = FastAPI()
run = False

@app.get("/")
async def read_root():
    return {"message": "Welcome to the API!"}

@app.get("/hello")
async def hello(name: Optional[str] = None):
    if name:
        return {"message": f"Hello, {name}!"}
    return {"message": "Hello, World!"}

def run_app():
    uvicorn.run(app, host="127.0.0.1", port=8000)

# Start the FastAPI app in a separate thread
if run:
    thread = threading.Thread(target=run_app)
    thread.start()

def get_greeting(name: Optional[str] = None):
    """
        GET request to /hello with optional 'name' parameter
    """
    base_url = "http://127.0.0.1:8000/hello"
    params = {}
    
    if name:
        params['name'] = name
        
    response = httpx.get(base_url, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": "Failed to retrieve data", "status_code": response.status_code}

if run:
    print(get_greeting())            # Output: {"message": "Hello, World!"}
    print(get_greeting("Alice"))     # Output: {"message": "Hello, Alice!"}


In [6]:
show_doc(get_greeting)

---

### get_greeting

>      get_greeting (name:Optional[str]=None)

GET request to /hello with optional 'name' parameter

In [7]:
with open('openapi.json', 'r') as f:
    openapi_spec = json.load(f)

tools = []

def process_schema(schema):
    if 'anyOf' in schema:
        return {"anyOf": [{"type": option['type'], "description": option.get('description', 'No description provided')} for option in schema['anyOf']]}
    else:
        return {"type": schema['type'], "description": schema.get('description', 'No description provided')}

for path, methods in openapi_spec['paths'].items():
    for method, details in methods.items():
        operation_id = details.get('operationId')
        description = details.get('summary', 'No description provided')

        parameters = details.get('parameters', [])

        param_properties = {}
        required_params = []

        for param in parameters:
            param_name = param['name']
            param_schema = param['schema']
            param_description = param.get('description', 'No description provided')

            param_properties[param_name] = process_schema(param_schema)

            if param.get('required', False):
                required_params.append(param_name)

        tool = {
            "type": "function",
            "function": {
                "name": operation_id,  # You can customize this
                "description": description,
                "parameters": {
                    "type": "object",
                    "properties": param_properties,
                    "required": required_params,
                    "additionalProperties": False,
                },
            }
        }

        tools.append(tool)

tools_json = json.dumps(tools, indent=2)

In [8]:
messages = [
    {"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."},
    {"role": "user", "content": "Hi, can you say hello to me?"}
]

# response = openai.chat.completions.create(
#     model="gpt-4o",
#     messages=messages,
#     tools=tools,
# )

In [9]:
#| hide
import nbdev; nbdev.nbdev_export()